Notebook utilizado para realizar a automatização da migração de histórias de um épico existente para um novo épico a ser criado pelo código abaixo.


É realizada a migração de histórias com status diferente de pronto.

In [ ]:
%pip install python-taiga

In [1]:
from taiga import TaigaAPI
import getpass
from taiga.models.models import UserStory
from taiga.requestmaker import RequestMaker

In [2]:
hostname = input("Informe o hostname: ")
if(not hostname or not hostname.strip()):
    raise Exception("Hostname não informado!")

token = getpass.getpass("Informe o token: ")
if(not token or not token.strip()):
    raise Exception("Token não informado!")


api = TaigaAPI(host=hostname,token=token)


# matricula = input("Informe a matrícula: ")
# senha = getpass.getpass("Informe a senha: ")

## Autenticação via LDAP no taiga
# api = TaigaAPI(
#     host= hostname,
#     auth_type='ldap'
# )
# api.auth(
#     username=matricula,
#     password=senha
# )


In [ ]:
api.me()

In [11]:
nome_projeto = input("Informe o nome do projeto: ")
if(not nome_projeto or not nome_projeto.strip()):
    raise Exception("Nome do projeto não informado!")

In [ ]:
lista_projetos = api.projects.list(q=nome_projeto)
for projetoItem in lista_projetos:
    print(f"\tProjeto: {projetoItem}")
    print(f"\t\tid: {projetoItem.id}")

print("\n")
projeto = None
if(lista_projetos):
    for projetoItem in lista_projetos:
        if(nome_projeto == projetoItem.name):
            projeto = projetoItem
            print(f"Projeto encontrado {projeto} - {projetoItem.id}")
            break
    if(not projeto):
        raise Exception("Projeto não encontrado! (1)")
else:
    raise Exception("Projeto não encontrado! (2)")

In [6]:
epico_entrada = input("Informe o nome do épico de entrada: ")
if(not epico_entrada or not epico_entrada.strip()):
    raise Exception("Épico de entrada não informado!")

In [28]:
epico_saida = input("Informe o nome do épico de saída: ")
if(not epico_saida or not epico_saida.strip()):
    raise Exception("Épico de saída não informado!")

In [ ]:
# search_result = api.search(projeto.id, epico_entrada)
# for epic_item in search_result.epics:
#     print (f"\t\tEpico: {epic_item}")

# epico_entrada_obj = None
# if(search_result.epics):
#     epico_entrada_obj = search_result.epics[0]
# else:
#     raise Exception("Épico não encontrado!")

Listando os épicos do projeto.

In [ ]:
lista_epicos = api.epics.list(project=projeto.id)
for epic_item in lista_epicos:
    print (f"\t\tEpico: {epic_item}")

Método para a busca de épico.

In [10]:
def coletar_epico_por_nome_em_lista(nome_epico="", lista_epicos=[]):
    epico_obj = None
    for epic_item in lista_epicos:
        if(nome_epico == epic_item.subject):
            print (f"\t\tEpico: {epic_item}")
            print (f"\t\t\tRef: {epic_item.ref}")
            print (f"\t\t\tSubject: {epic_item.subject}")
            epico_obj = projeto.get_epic_by_ref(epic_item.ref)
            print (f"\t\t\tDescription: {epico_obj.description}")
            print (f"\t\t\tAttributes: {epico_obj.get_attributes()}")
            print (f"\t\t\tto_dict(): {epico_obj.to_dict()}")
    return epico_obj

In [ ]:
epico_entrada_obj = coletar_epico_por_nome_em_lista(nome_epico=epico_entrada, lista_epicos=lista_epicos)
if(not epico_entrada_obj):
    raise Exception("Épico não encontrado!")

In [29]:
epico_saida_obj = coletar_epico_por_nome_em_lista(nome_epico=epico_saida, lista_epicos=lista_epicos)
if(epico_saida_obj):
    raise Exception("Épico de saída já existe, é necessário informar o nome de um épico novo.")

In [ ]:
lista_historias_epico_entrada = epico_entrada_obj.list_user_stories()
for historia in lista_historias_epico_entrada:
    print(type(historia))
    print(f"\tHistoria: {historia.id}")
    print(f"\t\tRef: {historia.ref}")
    print(f"\t\tProject: {historia.project}")
    print(f"\t\ttatus: {historia.status}")
    print(f"\t\ttatus: {historia.status_extra_info}")
    print(f"\t\tSubject: {historia.subject}")
    print(f"\t\tAtributos: {historia.get_attributes()}")
    historia2 = projeto.get_userstory_by_ref(historia.ref)
    print(f"\t\tDescription: {historia2.description}")
    print(historia.to_dict())
    print(historia2.to_dict())

    # for tarefa in historia2.list_tasks():
    #     tarefa2 = projeto.get_task_by_ref(tarefa.ref)
    #     print(f"\t\tTarefa: {tarefa2.id}")
    #     print(f"\t\t\tRef: {tarefa2.ref}")
    #     print(f"\t\t\tSubject: {tarefa2.subject}")
    #     print(f"\t\t\tDescription: {tarefa2.description}")
    #     print(f"\t\t\tAtributos: {tarefa2.get_attributes()}")
    #     print(f"\t\t\tDict: {tarefa2.to_dict()}")
        

    print("----")


Método para a cópia de história.

In [15]:
def copiar_historia(historia_origem=None):
    if not historia_origem:
        raise Exception("É necessário passar a história de origem para a cópia.")

    historia_tmp = api.user_stories.create(
                                assigned_to=historia_origem.assigned_to, 
                                assigned_users=None, 
                                backlog_order=historia_origem.backlog_order, 
                                version=historia_origem.version, 
                                client_requirement=historia_origem.client_requirement, 
                                description=historia_origem.description, 
                                is_blocked=historia_origem.is_blocked, 
                                kanban_order=historia_origem.kanban_order, 
                                milestone=historia_origem.milestone, 
                                points=historia_origem.points, 
                                project=historia_origem.project, 
                                sprint_order=historia_origem.sprint_order, 
                                status=historia_origem.status, 
                                subject=historia_origem.subject, 
                                tags=historia_origem.tags, 
                                team_requirement=historia_origem.team_requirement, 
                                watchers=historia_origem.watchers, 
                                due_date=None, 
                                generated_from_issue=historia_origem.generated_from_issue, 
                                generated_from_task=None)

    if 'attributes_values' in historia_origem.get_attributes():
        for atributo, valor in historia_origem.get_attributes()['attributes_values'].items():
            print(f"atributo: {atributo}, valor: {valor}")
            historia_tmp.set_attribute(atributo, valor, 1)




In [ ]:
# Loop de cópia das histórias
# for historia in lista_historias_epico_entrada:
#     historia2 = projeto.get_userstory_by_ref(historia.ref)
#     copiar_historia(historia_origem=historia2)

Criação do Épico de destino

In [30]:
if not epico_saida:
    raise Exception("Nome do épico de saída não informado!")


epico_saida_obj = api.epics.create(
                                assigned_to=epico_entrada_obj.assigned_to, 
                                blocked_note=epico_entrada_obj.blocked_note, 
                                description=epico_entrada_obj.description, 
                                is_blocked=False, 
                                is_closed=False,
                                color=epico_entrada_obj.color, 
                                project=epico_entrada_obj.project, 
                                subject=epico_saida, 
                                tags=epico_entrada_obj.tags, 
                                watchers=epico_entrada_obj.watchers, 
                                version=epico_entrada_obj.version)

if 'attributes_values' in epico_entrada_obj.get_attributes():
    for atributo, valor in epico_entrada_obj.get_attributes()['attributes_values'].items():
        print(f"atributo: {atributo}, valor: {valor}")
        epico_saida_obj.set_attribute(atributo, valor, 1)


Varredura das histórias com status difetente de pronto.

In [ ]:
lista_user_histories_diferentes_pronta = []

for historia in lista_historias_epico_entrada:
    print(type(historia))
    print(f"\tHistoria: {historia.id}")
    print(f"\t\tRef: {historia.ref}")
    print(f"\t\tProject: {historia.project}")
    print(f"\t\ttatus: {historia.status}")
    print(f"\t\ttatus: {historia.status_extra_info}")

    if historia.status_extra_info['name'] not in ['Ready',]:
        lista_user_histories_diferentes_pronta.append(historia)

Vinculando as histórias selecionadas ao épico de saída.

In [32]:
request_maker_obj = RequestMaker('/api/v1/', hostname, token)
for historia_tmp in lista_user_histories_diferentes_pronta:
    mapa_payload = {}
    mapa_payload['epic']=epico_saida_obj.id
    mapa_payload['user_story']=historia_tmp.id
    
    # Criando o vínculo com o novo épico.
    request_maker_obj.post(f'epics/{epico_saida_obj.id}/related_userstories', payload=mapa_payload)

    # Desvinculando ao épico anterior.
    # request_maker_obj.delete(f'epics/{epico_entrada_obj.id}/related_userstories/{historia_tmp.id}')
